In [2]:
import pandas as pd
import networkx as nx
import pickle
from tqdm.notebook import tqdm

In [3]:
path_1 = "./folder_05/"
file_mesh = 'mesh_list_2021.pickle'
file_mesh_11 = 'mesh_list_2011.pickle'
file_hsdn = "hsdn.csv"
sup_3 = "Supplementary Data 3.txt"
file_co_occ = "co_occ_pair.csv"#カット前の全共起ペア
file_cutoff = "cutoff_pair.csv"#カット後の共起ペア

In [6]:
sup_3_df = pd.read_table(sup_3, engine="python")
print(sup_3_df.columns)
print(len(sup_3_df))
sup_3_df[:3]
#Supplementary Data 3.txtは、全ペア（カットせず）に対しTFIDF scoreを出している？
#ペアの数147,978あるが、カイ二乗検定を用いてカットする前の全ペアの数.
#カットした後のペアの数は 62,820とZhou本文に記載あり

Index(['MeSH Symptom Term', 'MeSH Disease Term', 'PubMed occurrence',
       'TFIDF score'],
      dtype='object')
147978


,MeSH Symptom Term,MeSH Disease Term,PubMed occurrence,TFIDF score
0,"Aging, Premature",Respiratory Syncytial Virus Infections,1,3.464551
1,"Aging, Premature",Orthomyxoviridae Infections,1,3.464551
2,"Aging, Premature",HIV Infections,3,10.393654


In [8]:
#HSDN　エッジリスト読込・ネットワーク作成
hsdn = pd.read_csv(path_1+file_hsdn)
print(len(hsdn), "\n")
G = nx.from_pandas_edgelist(hsdn, "disease_1", "disease_2" )
print(nx.info(G))

7804331 

Name: 
Type: Graph
Number of nodes: 4716
Number of edges: 7804331
Average degree: 3309.7248


In [9]:
#疾患リンク保有数
ranking = pd.concat( [hsdn["disease_1"], hsdn["disease_2"] ], axis=0)#7804331 *2 = 15608662
ranking.value_counts()

Intellectual Disability             4709
Pregnancy Complications             4706
Hypertension                        4706
Obesity                             4702
Epilepsy                            4701
                                    ... 
Piedra                               121
Childhood-Onset Fluency Disorder      99
Vaccine-Preventable Diseases          98
Narcotic-Related Disorders            35
Historical Trauma                     34
Length: 4716, dtype: int64

In [10]:
#ネットワークの疾患MeSHを比較
#アップデートして消えた疾患MeSH、登場した疾患MeSHを調査

In [11]:
#MeSH

#2021_MeSH
with open(file_mesh, 'rb') as f:
    mesh_list = pickle.load(f)
#Zhouらの定義した疾患MeSH
all_disease = [descriptor for descriptor in mesh_list for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
#重複削除
disease_mesh = []
[dm for dm in all_disease if dm not in disease_mesh and not disease_mesh.append(dm)]

print("all_mesh:", len(mesh_list), "disease_mesh:", len(disease_mesh))

all_mesh: 29917 disease_mesh: 4887


In [12]:
#2011_MeSH
with open(file_mesh_11, 'rb') as f:
    mesh_list_11 = pickle.load(f)
#Zhouらの定義した疾患MeSH
all_disease_11 = [descriptor for descriptor in mesh_list_11 for uid in descriptor[-1] if ((uid.startswith("C")==True) & (uid.startswith("C22")!=True)&(uid.startswith("C23.888")!=True)) | (uid.startswith("F03")==True) ]
#重複削除
disease_mesh_11 = []
[dm_11 for dm_11 in all_disease_11 if dm_11 not in disease_mesh_11 and not disease_mesh_11.append(dm_11)]
print("all_mesh_11:", len(mesh_list_11), "disease_mesh_11:", len(disease_mesh_11))

all_mesh_11: 26142 disease_mesh_11: 4475


In [13]:
#ネットワークの疾患MeSH

#zhouのHSDNの疾患MeSH :  zhouのSupplementary Data 3.txt（症状共起ペアのTFIDFデータ）の疾患MeSHを用いた
zhou_disease_df = pd.read_table(sup_3, engine="python")["MeSH Disease Term"]
zhou_disease = zhou_disease_df.unique().tolist()

#アップデートしたHSDNの疾患MeSH
hsdn_disease = ranking.unique().tolist()

print(len(zhou_disease), len(hsdn_disease))

4219 4716


In [14]:
#MeSHの名前が変更しているかもしれないためIDにしておく
zhou_id = list(set([ z_s[0] for z_i in zhou_disease for z_s in disease_mesh_11  if z_i == z_s[1] ]))
hsdn_id = list(set([ s[0] for i in hsdn_disease for s in disease_mesh  if i == s[1] ]))
print(len(zhou_id),len(hsdn_id))

4218 4716


In [15]:
#zhou_id　とhsdn_id　を比較. 差分や共通要素となるIDを取得
common = list(set(zhou_id)& set(hsdn_id))
only_zhou = list(set(zhou_id)- set(hsdn_id))
only_hsdn = list(set(hsdn_id)- set(zhou_id))
print("zhou, hsdn共通疾患MeSH_ID:", len(common))
print("zhouのみにある疾患MeSH_ID:", len(only_zhou))
print("hsdnのみにある疾患MeSH_ID:", len(only_hsdn))

zhou, hsdn共通疾患MeSH_ID: 4194
zhouのみにある疾患MeSH_ID: 24
hsdnのみにある疾患MeSH_ID: 522


In [16]:
#差分
def compare(a_list, b_list):
    col_name = ["id", "disease"]
    a = pd.DataFrame(a_list, columns=col_name)    #2011MeSHにあるもの
    b = pd.DataFrame(b_list, columns=col_name)    #2021MeSHにあるもの
    c = pd.merge(a, b, on=col_name)    #2011, 2021どちらにもあるもの
    
    d = pd.concat([a,c], axis=0)
    d_ = d.drop_duplicates(keep=False)    #2011にしかないもの
    
    e = pd.concat([b,c], axis=0) 
    e_ = e.drop_duplicates(keep=False)   #2021にしかないもの
    
    print("In 2011MeSH: ", len(a))
    print("In 2021MeSH: ",len(b))
    print("In 2011&2021MeSH:", len(c))
    print("2011MeSHにしかないもの:", len(d_))
    print("2021MeSHにしかないもの:",len(e_))
    print()

In [17]:
# そのネットワークにしかない疾患MeSH_IDから、IDに対応するDescriptor名をMeSHデータから取得
def network_only_mesh(network_name, a_list, b_list):
    a_list=[]
    b_list=[]
    for i in network_name:
        for s in disease_mesh_11: #MeSH2011
            if i == s[0]:
                a_list.append([s[0],s[1]])
                break
        for s in disease_mesh: #2021MeSH
            if i == s[0]:
                b_list.append([s[0],s[1]])
                break
    a_list =  sorted(a_list, key=lambda x: x[0])
    b_list =  sorted(b_list, key=lambda x: x[0])
    
    compare(a_list, b_list)

In [18]:
print("only_zhou:")
network_only_mesh(only_zhou, "only_zhou_with_name_11", "only_zhou_with_name_21")
print("only_hsdn:")
network_only_mesh(only_hsdn, "only_hsdn_with_name_11", "only_hsdn_with_name_21")

only_zhou:
In 2011MeSH:  24
In 2021MeSH:  2
In 2011&2021MeSH: 2
2011MeSHにしかないもの: 22
2021MeSHにしかないもの: 0

only_hsdn:
In 2011MeSH:  169
In 2021MeSH:  522
In 2011&2021MeSH: 168
2011MeSHにしかないもの: 1
2021MeSHにしかないもの: 354



#hsdnのみにある疾患MeSH_IDに対応するDescriptor名を取得
only_hsdn_with_name_11 = []
only_hsdn_with_name_21 = []

for i in only_hsdn:
    for s in disease_mesh_11:#2011MeSH
        if i == s[0]:
            only_hsdn_with_name_11.append([s[0],s[1]])
            break;
    for s in disease_mesh:#2021MeSH
        if i == s[0]:
            only_hsdn_with_name_21.append([s[0],s[1]])
            break;
only_hsdn_with_name_11 =  sorted(only_hsdn_with_name_11, key=lambda x: x[0])
only_hsdn_with_name_21 =  sorted(only_hsdn_with_name_21, key=lambda x: x[0])


def compare(name_11, name_12):
    col_name = ["id", "disease"]
    a = pd.DataFrame(name_11, columns=col_name)    #2011MeSHにあるもの
    b = pd.DataFrame(name_12, columns=col_name)    #2021MeSHにあるもの
    c = pd.merge(a, b, on=col_name)    #2011, 2021どちらにもあるもの
    d = pd.concat([a,c], axis=0)
    d_ = d.drop_duplicates(keep=False)    #2011にしかないもの
    e = pd.concat([b,c], axis=0) 
    e_ = e.drop_duplicates(keep=False)   #2021にしかないもの
    print("In 2011MeSH: ", len(a))
    print("In 2021MeSH: ",len(b))
    print("In 2011&2021MeSH:", len(c))
    print("2011MeSHにしかないもの:", len(d_))
    print("2021MeSHにしかないもの:",len(e_))
    print()

print("**only_zhou:", len(only_zhou))
compare(only_zhou_with_name_11, only_zhou_with_name_21)
print("\n","**only_hsdn:", len(only_hsdn))
compare(only_hsdn_with_name_11, only_hsdn_with_name_21)

In [19]:
#MeSHそのものについて調査
#2011MeSH→2021MeSHで変更があったか調べる

In [20]:
disease_id_21 = [i[0] for i in disease_mesh]
disease_id_11 = [i_11[0] for i_11 in disease_mesh_11]
print("2021_疾患MeSHの数:", len(disease_id_21))
print("2011_疾患MeSHの数:", len(disease_id_11), "\n")

common = list(set(disease_id_21)& set(disease_id_11))
only_21 = list(set(disease_id_21)- set(disease_id_11))
only_11 = list(set(disease_id_11)- set(disease_id_21))
print("2011, 2021同じ:", len(common))
print("2021でのみ見られるID:", len(only_21))
print("2011でのみ見られるID:", len(only_11))

2021_疾患MeSHの数: 4887
2011_疾患MeSHの数: 4475 

2011, 2021同じ: 4451
2021でのみ見られるID: 436
2011でのみ見られるID: 24


In [21]:
def network_has_mesh(name, c_list, disease_mesh_version):
    for i in name:
        for s in disease_mesh_version: #MeSH2011
            if i == s[0]:
                c_list.append([s[0],s[1]])
                break
    #c_list =  sorted(c_list, key=lambda x: x[0])
    print(len(c_list), c_list[:3])

#2011にしかないMeSHの詳細（2021では見られなかった疾患MeSH)
print("only_11:")
disappeared=[]
network_has_mesh(only_11, disappeared, disease_mesh_11)

#2021にしかないMeSHの詳細（2011では見られなかった疾患MeSH)
print("only_21:")
appeared=[]
network_has_mesh(only_21, appeared, disease_mesh)

only_11:
24 [['D004816', 'Epidermal Necrolysis, Toxic'], ['D048949', 'Labor Pain'], ['D001787', 'Blood Group Incompatibility']]
only_21:
436 [['D000077299', 'Healthcare-Associated Pneumonia'], ['D000071297', 'Acute Retroviral Syndrome'], ['D000079690', 'Postoperative Cognitive Complications']]


In [22]:
#新しく登場したMeSHのうち最近のもの（桁が多い：10(D＋9)桁、　これまでは7(D＋6)桁）を取得
new_disease = sorted([i for i in appeared  if len(i[0])>7 ], reverse=True)
print(len(new_disease))
print("example:", new_disease[:5])

254
example: [['D000087122', 'Mania'], ['D000086982', 'Blood-Borne Infections'], ['D000086966', 'Bacterial Zoonoses'], ['D000086965', 'Viral Zoonoses'], ['D000086582', 'Anosmia']]


In [27]:
#2011MeSH
[ term for term in disease_id_11 if len(term[0])>7 ]

[]

In [23]:
co_occ = pd.read_csv(path_1+file_co_occ)#カット前ペア
cutoff = pd.read_csv(path_1+file_cutoff)#カット後ペア

In [24]:
#新しい疾患のうち, 症状と共起しているものはあるか. 
#ペアカット前後の共起件数

target_new_disease_list = []
for target_d in new_disease:
    target_new_disease_list.append([target_d[0], target_d[1], len(co_occ[co_occ["disease"]==target_d[1]]), len(cutoff[cutoff["disease"]==target_d[1]])])
target_new_disease_df = pd.DataFrame(target_new_disease_list, 
                                     columns=["id", 
                                              "new_disease", 
                                              "co_occ_num(before_cut)", 
                                              "co_occ_num(after_cut)"])  

In [25]:
target_new_disease_df
#D000067011 --> Date of Entry: 2015/07/01, Revision Date: 2014/10/15
#https://meshb.nlm.nih.gov/record/ui?ui=D000067011

,id,new_disease,co_occ_num(before_cut),co_occ_num(after_cut)
0,D000087122,Mania,0,0
1,D000086982,Blood-Borne Infections,0,0
2,D000086966,Bacterial Zoonoses,0,0
3,D000086965,Viral Zoonoses,0,0
4,D000086582,Anosmia,0,0
...,...,...,...,...
249,D000067329,"Obesity, Metabolically Benign",15,10
250,D000067251,Symptom Flare Up,17,16
251,D000067208,Shellfish Hypersensitivity,0,0
252,D000067073,Psychological Trauma,33,17


In [28]:
target_new_disease_df[target_new_disease_df["new_disease"]=="COVID-19"]

,id,new_disease,co_occ_num(before_cut),co_occ_num(after_cut)
5,D000086382,COVID-19,0,0


target_new_disease_df.to_csv("new_disease.csv", index=False)